## 4D CT Performance in SPECT/CT-negative Patients

This section analyzes patients whose **SPECT/CT scans were non-localizing** but **4D CT scans localized pathology**, evaluating the distribution of concordance outcomes.

The table below summarizes the **counts and percentages** for each match category among these SPECT-negative, 4D CT-positive cases.

In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [ ]:
# Define modalities
spect_modality = 'SPECT/CT'
modality_4dct = '4D CT Scan'

# Filter for SPECT non-localizing cases
negative_spect = preop_imaging_df[preop_imaging_df[spect_modality] == 'Non-localizing'].copy()

# Filter for those with positive 4D CT localization (Left or Right side)
spect_neg_4d_pos = negative_spect[
    negative_spect['4D CT Imaging Side'].isin(['Left', 'Right'])
].copy()

# Calculate counts after creating the column
exact_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Exact)').sum()
# ... rest of count calculations remain unchanged

In [3]:
# Calculate counts for match types
total_cases = spect_neg_4d_pos.shape[0]
exact_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Exact)').sum()
side_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Side)').sum()
partial_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Yes (Partial)').sum()
incorrect_count = (spect_neg_4d_pos['4D CT Scan Calculated Concordance'] == 'Incorrect Localization').sum()

# Calculate percentages
exact_pct = (exact_count / total_cases * 100) if total_cases > 0 else None
side_pct = (side_count / total_cases * 100) if total_cases > 0 else None
partial_pct = (partial_count / total_cases * 100) if total_cases > 0 else None
incorrect_pct = (incorrect_count / total_cases * 100) if total_cases > 0 else None

# # Calculate combined side match
# total_side_match_count = exact_count + side_count
# total_side_match_pct = (total_side_match_count / total_cases * 100) if total_cases > 0 else None


In [4]:
# Create clean final summary dataframe
spect_neg_4d_pos_summary_df = pd.DataFrame({
    'Match Type': [
        'Exact Match',
        'Side Match',
        'Partial Match',
        'Incorrect Localization'
    ],
    'Count': [
        exact_count,
        side_count,
        partial_count,
        incorrect_count
    ],
    'Percentage': [
        f"{exact_pct:.2f}%" if exact_pct is not None else 'N/A',
        f"{side_pct:.2f}%" if side_pct is not None else 'N/A',
        f"{partial_pct:.2f}%" if partial_pct is not None else 'N/A',
        f"{incorrect_pct:.2f}%" if incorrect_pct is not None else 'N/A'
    ]
})

# Display results
print(f"Total SPECT-negative / 4D CT-positive cases: {total_cases}")
display(spect_neg_4d_pos_summary_df)

Total SPECT-negative / 4D CT-positive cases: 31


,Match Type,Count,Percentage
0,Exact Match,15,48.39%
1,Side Match,4,12.90%
2,Partial Match,1,3.23%
3,Incorrect Localization,11,35.48%
